In [1]:
import torch
import torch.nn.functional as F
from src.utils import *
from src.models.models import Model_Phrase_Concatenation, Model_Phrase_Extraction, Model_concat_nopooling
from src.models.baseline import Baseline
from datasets import concatenate_datasets
from sklearn.utils.class_weight import compute_class_weight
from transformers import DefaultDataCollator, AutoTokenizer, TrainingArguments, Trainer

c:\Users\marco\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
RANDOM_SEED = 42
set_seeds(RANDOM_SEED)

print(torch.__version__)
print(torch.cuda.is_available())

2.1.0+cu121
True


In [3]:
url = "https://drive.google.com/uc?export=download&id=1wVNU2XvvhqjaGXZM-JLJwOt97gt4g9j2"
dataset_name = "MELD_train_efr.json"

df_manager = DataframeManager(url, dataset_name)

df = df_manager.produce_df()
df

Current work directory: c:\Users\marco\OneDrive\Immagini\Documenti\GitHub\ediref


,episode,emotions,utterances,triggers,emotions_id
0,utterance_0,"[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0, 0, 0, 1, 0]","[4, 4, 4, 4, 3]"
1,utterance_1,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 1, 0]","[4, 4, 4, 4, 3, 4, 4]"
2,utterance_2,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 1]"
3,utterance_3,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 1, 4, 3]"
4,utterance_4,"[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0, 0, 1, 0]","[3, 0, 3, 1]"
...,...,...,...,...,...
3995,utterance_3995,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[4, 6, 4, 4, 3, 2, 4, 2, 3, 4, 4, 2]"
3996,utterance_3996,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[4, 6, 4, 4, 3, 2, 4, 2, 3, 4, 4, 2, 2, 4]"
3997,utterance_3997,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[4, 6, 4, 4, 3, 2, 4, 2, 3, 4, 4, 2, 2, 4, 4]"
3998,utterance_3998,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[4, 6, 4, 4, 3, 2, 4, 2, 3, 4, 4, 2, 2, 4, 4, 3]"


In [4]:
model_card = 'google/electra-base-discriminator'

tokenizer = AutoTokenizer.from_pretrained(model_card)

model_dir = "./model_dir/"+model_card.split("/")[-1]+"/"
data_collator = DefaultDataCollator()

In [5]:
train_data_tokenized, val_data_tokenized, test_data_tokenized = df_manager.produce_dataset(tokenizer, RANDOM_SEED)

In [6]:
len(train_data_tokenized)

28062

In [7]:
def init_pos_weight(data, labels_column, class_weights=True, factor=1):
    y = data[labels_column].numpy()
    if class_weights:
        return torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)).to("cuda")
    else:
        return torch.tensor(compute_class_weight(class_weight=None, classes=np.unique(y), y=y)).to("cuda")

In [8]:
class MultiLabelTrainer(Trainer):
    def __init__(self, pos_weight=None, **kwargs):
        self.emotions_pos_weight, self.triggers_pos_weight = pos_weight
        super().__init__(**kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        emotions_true = inputs["emotions_id_one_hot_encoding"].to("cuda")
        triggers_true = inputs["triggers"].unsqueeze(1).float().to("cuda")

        result = model(**inputs)
        
        emotion_logits = result['emotion_logits'].to("cuda")
        trigger_logits = result['trigger_logits'].to("cuda")
        
        loss_fct_emotions = torch.nn.CrossEntropyLoss(weight=self.emotions_pos_weight).to("cuda")        
        loss_fct_triggers = torch.nn.BCEWithLogitsLoss(pos_weight=self.triggers_pos_weight[1]).to("cuda")
        
        loss_emotions = loss_fct_emotions(emotion_logits, emotions_true.float())
        loss_triggers = loss_fct_triggers(trigger_logits, triggers_true)


        loss_emotions_wt = 0.5
        loss_triggers_wt = 0.5

        loss = loss_emotions_wt*loss_emotions + loss_triggers_wt*loss_triggers
        return (loss, {'emotion_logits': emotion_logits, 'trigger_logits': trigger_logits}) if return_outputs else loss

def get_trainer(model, train, val, model_dir, class_weights=True, batch_size=1, epochs=20):
    training_args = TrainingArguments(
        output_dir=model_dir,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        lr_scheduler_type="cosine",
        metric_for_best_model='u_avg_f1',
        report_to='none',
        label_names=["emotions_id", "triggers", "dialogue_index"],
    )

    full_dataset = concatenate_datasets([train_data_tokenized, val_data_tokenized, test_data_tokenized])
    pos_weight = (init_pos_weight(full_dataset, df_manager.column_emotions_id, class_weights), init_pos_weight(full_dataset, df_manager.column_triggers, class_weights))

    trainer = MultiLabelTrainer(
        pos_weight=pos_weight,
        model=model,
        args=training_args,
        train_dataset=train,
        eval_dataset=val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=lambda pred: compute_metrics(pred, len(df_manager.emotion2id.keys()))
    )

    return trainer

In [8]:
# How to load a model
# b = Model_Phrase_Extraction(len(df_manager.unique_emotions), tokenizer.sep_token_id)
# model_path = model_dir + "bert_extraction_51"
# b.load_state_dict(torch.load(model_path+"/bert_extraction_51.pth"))
# tr = get_trainer(b, train_data_tokenized, val_data_tokenized, model_path, class_weights=True, batch_size=4, epochs=5)
# tr.evaluate(val_data_tokenized)

In [11]:
def train_extr_model(seed, freeze=False):
    set_seeds(seed)
    base_model = Model_Phrase_Extraction(len(df_manager.unique_emotions), tokenizer.sep_token_id, freeze)
    model_path = model_dir+"electra_extraction_freeze_"+str(seed) if freeze else model_dir+"electra_extraction_"+str(seed)

    trainer = get_trainer(base_model, train_data_tokenized, val_data_tokenized, model_path, class_weights=True, batch_size=4, epochs=5)

    print(f'Training EXTRACTION MODEL with seed {seed}:')

    trainer.train()
    save_name = "/electra_extraction_freeze_"+str(seed)+".pth" if freeze else "/electra_extraction_"+str(seed)+".pth"
    torch.save(base_model.state_dict(), model_path+save_name)

In [10]:
seeds = [49, 666, 51, 77, 111]

In [18]:
# Training model with seed 49
train_extr_model(49)

c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
c:\Users\marco\anaconda3\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Training EXTRACTION MODEL with seed 49:


  0%|          | 0/35080 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.5421, 'learning_rate': 1.9989976531650042e-05, 'epoch': 0.07}
{'loss': 1.3779, 'learning_rate': 1.9959926220583713e-05, 'epoch': 0.14}
{'loss': 1.3744, 'learning_rate': 1.9909909308469398e-05, 'epoch': 0.21}
{'loss': 1.3762, 'learning_rate': 1.9840026063894193e-05, 'epoch': 0.29}
{'loss': 1.3867, 'learning_rate': 1.9750416581356147e-05, 'epoch': 0.36}
{'loss': 1.3281, 'learning_rate': 1.9641260500417672e-05, 'epoch': 0.43}
{'loss': 1.3626, 'learning_rate': 1.9512776645583266e-05, 'epoch': 0.5}
{'loss': 1.3262, 'learning_rate': 1.9365222587623407e-05, 'epoch': 0.57}
{'loss': 1.3487, 'learning_rate': 1.9198894127224075e-05, 'epoch': 0.64}
{'loss': 1.2573, 'learning_rate': 1.9014124701996973e-05, 'epoch': 0.71}
{'loss': 1.2816, 'learning_rate': 1.881128471803926e-05, 'epoch': 0.78}
{'loss': 1.2973, 'learning_rate': 1.859078080738279e-05, 'epoch': 0.86}
{'loss': 1.2123, 'learning_rate': 1.835305501282148e-05, 'epoch': 0.93}
{'loss': 1.2163, 'learning_rate': 1.809858390175087e-05

...

{'eval_loss': 0.8383831019836456, 'eval_accuracy_emotions': 0.6805, 'eval_accuracy_triggers': 0.7981, 'eval_u_avg_f1': 0.6634, 'eval_u_f1scores_emotions': 0.6314, 'eval_u_f1scores_triggers': 0.6955, 'eval_d_f1scores_emotions': 0.6662, 'eval_d_f1scores_triggers': 0.7552, 'eval_runtime': 688.1009, 'eval_samples_per_second': 4.995, 'eval_steps_per_second': 1.25, 'epoch': 3.0}
{'loss': 0.7012, 'learning_rate': 6.527489286891458e-06, 'epoch': 3.06}
{'loss': 0.6537, 'learning_rate': 6.111198874944846e-06, 'epoch': 3.14}
{'loss': 0.6662, 'learning_rate': 5.702704317997491e-06, 'epoch': 3.21}
{'loss': 0.6652, 'learning_rate': 5.302824522501931e-06, 'epoch': 3.28}
{'loss': 0.6859, 'learning_rate': 4.912361124952949e-06, 'epoch': 3.35}
{'loss': 0.6768, 'learning_rate': 4.532096884851978e-06, 'epoch': 3.42}
{'loss': 0.6672, 'learning_rate': 4.162794115514078e-06, 'epoch': 3.49}
{'loss': 0.6466, 'learning_rate': 3.805193155863247e-06, 'epoch': 3.56}
{'loss': 0.6467, 'learning_rate': 3.460010886279

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.7778449919411695, 'eval_accuracy_emotions': 0.7966, 'eval_accuracy_triggers': 0.8248, 'eval_u_avg_f1': 0.7354, 'eval_u_f1scores_emotions': 0.7767, 'eval_u_f1scores_triggers': 0.6941, 'eval_d_f1scores_emotions': 0.7866, 'eval_d_f1scores_triggers': 0.7868, 'eval_runtime': 688.1652, 'eval_samples_per_second': 4.994, 'eval_steps_per_second': 1.25, 'epoch': 4.0}
{'loss': 0.5893, 'learning_rate': 1.6865479389478545e-06, 'epoch': 4.06}
{'loss': 0.5765, 'learning_rate': 1.4461099843816684e-06, 'epoch': 4.13}
{'loss': 0.6095, 'learning_rate': 1.2228199589835998e-06, 'epoch': 4.2}
{'loss': 0.5745, 'learning_rate': 1.0171254908541373e-06, 'epoch': 4.28}
{'loss': 0.5515, 'learning_rate': 8.294389343914899e-07, 'epoch': 4.35}
{'loss': 0.5725, 'learning_rate': 6.601365436473439e-07, 'epoch': 4.42}
{'loss': 0.5594, 'learning_rate': 5.095577180527378e-07, 'epoch': 4.49}
{'loss': 0.5649, 'learning_rate': 3.780043220261764e-07, 'epoch': 4.56}
{'loss': 0.5594, 'learning_rate': 2.657400798

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.7620718345346947, 'eval_accuracy_emotions': 0.8004, 'eval_accuracy_triggers': 0.8208, 'eval_u_avg_f1': 0.7363, 'eval_u_f1scores_emotions': 0.7827, 'eval_u_f1scores_triggers': 0.6899, 'eval_d_f1scores_emotions': 0.7851, 'eval_d_f1scores_triggers': 0.7805, 'eval_runtime': 689.194, 'eval_samples_per_second': 4.987, 'eval_steps_per_second': 1.248, 'epoch': 5.0}
{'train_runtime': 11083.7652, 'train_samples_per_second': 12.659, 'train_steps_per_second': 3.165, 'train_loss': 0.9017104922026022, 'epoch': 5.0}


In [12]:
# Training model with seed 666
train_extr_model(666)

c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Training EXTRACTION MODEL with seed 666:


  0%|          | 0/35080 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.5455, 'grad_norm': 17.883642196655273, 'learning_rate': 1.9989976531650042e-05, 'epoch': 0.07}
{'loss': 1.4613, 'grad_norm': 6.613398551940918, 'learning_rate': 1.9959926220583713e-05, 'epoch': 0.14}
{'loss': 1.3438, 'grad_norm': 3.1198196411132812, 'learning_rate': 1.9909909308469398e-05, 'epoch': 0.21}
{'loss': 1.3474, 'grad_norm': 15.932950973510742, 'learning_rate': 1.9840026063894193e-05, 'epoch': 0.29}
{'loss': 1.3518, 'grad_norm': 4.3627848625183105, 'learning_rate': 1.9750416581356147e-05, 'epoch': 0.36}
{'loss': 1.3196, 'grad_norm': 1.7432825565338135, 'learning_rate': 1.9641260500417672e-05, 'epoch': 0.43}
{'loss': 1.3413, 'grad_norm': 7.337068557739258, 'learning_rate': 1.9512776645583266e-05, 'epoch': 0.5}
{'loss': 1.2773, 'grad_norm': 26.30140495300293, 'learning_rate': 1.9365222587623407e-05, 'epoch': 0.57}
{'loss': 1.2863, 'grad_norm': 5.915427207946777, 'learning_rate': 1.9198894127224075e-05, 'epoch': 0.64}
{'loss': 1.286, 'grad_norm': 7.643270969390869, 'le

...

{'eval_loss': 0.8041666043396103, 'eval_accuracy_emotions': 0.7349, 'eval_accuracy_triggers': 0.8187, 'eval_u_avg_f1': 0.6946, 'eval_u_f1scores_emotions': 0.6871, 'eval_u_f1scores_triggers': 0.7022, 'eval_d_f1scores_emotions': 0.7177, 'eval_d_f1scores_triggers': 0.7828, 'eval_runtime': 688.3264, 'eval_samples_per_second': 4.993, 'eval_steps_per_second': 1.249, 'epoch': 3.0}
{'loss': 0.6828, 'grad_norm': 18.10382652282715, 'learning_rate': 6.527489286891458e-06, 'epoch': 3.06}
{'loss': 0.7285, 'grad_norm': 9.364639282226562, 'learning_rate': 6.111198874944846e-06, 'epoch': 3.14}
{'loss': 0.6343, 'grad_norm': 28.35895538330078, 'learning_rate': 5.702704317997491e-06, 'epoch': 3.21}
{'loss': 0.6462, 'grad_norm': 8.710488319396973, 'learning_rate': 5.302824522501931e-06, 'epoch': 3.28}
{'loss': 0.6763, 'grad_norm': 17.407514572143555, 'learning_rate': 4.912361124952949e-06, 'epoch': 3.35}
{'loss': 0.6236, 'grad_norm': 37.24601745605469, 'learning_rate': 4.532096884851978e-06, 'epoch': 3.42

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.7298704705726259, 'eval_accuracy_emotions': 0.8086, 'eval_accuracy_triggers': 0.8167, 'eval_u_avg_f1': 0.7381, 'eval_u_f1scores_emotions': 0.7775, 'eval_u_f1scores_triggers': 0.6987, 'eval_d_f1scores_emotions': 0.7987, 'eval_d_f1scores_triggers': 0.7743, 'eval_runtime': 688.6431, 'eval_samples_per_second': 4.991, 'eval_steps_per_second': 1.249, 'epoch': 4.0}
{'loss': 0.5512, 'grad_norm': 8.321831703186035, 'learning_rate': 1.6865479389478545e-06, 'epoch': 4.06}
{'loss': 0.565, 'grad_norm': 35.19001007080078, 'learning_rate': 1.4461099843816684e-06, 'epoch': 4.13}
{'loss': 0.5447, 'grad_norm': 8.165038108825684, 'learning_rate': 1.2228199589835998e-06, 'epoch': 4.2}
{'loss': 0.5972, 'grad_norm': 16.562562942504883, 'learning_rate': 1.0171254908541373e-06, 'epoch': 4.28}
{'loss': 0.5549, 'grad_norm': 11.985882759094238, 'learning_rate': 8.294389343914899e-07, 'epoch': 4.35}
{'loss': 0.5572, 'grad_norm': 10.403803825378418, 'learning_rate': 6.601365436473439e-07, 'epoch': 

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.7408899538003193, 'eval_accuracy_emotions': 0.8179, 'eval_accuracy_triggers': 0.826, 'eval_u_avg_f1': 0.7428, 'eval_u_f1scores_emotions': 0.7958, 'eval_u_f1scores_triggers': 0.6899, 'eval_d_f1scores_emotions': 0.8085, 'eval_d_f1scores_triggers': 0.7841, 'eval_runtime': 688.0756, 'eval_samples_per_second': 4.995, 'eval_steps_per_second': 1.25, 'epoch': 5.0}
{'train_runtime': 10956.4467, 'train_samples_per_second': 12.806, 'train_steps_per_second': 3.202, 'train_loss': 0.8841403148872958, 'epoch': 5.0}


In [13]:
# Training model with seed 51
train_extr_model(51)

c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Training EXTRACTION MODEL with seed 51:


  0%|          | 0/35080 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.5555, 'grad_norm': 18.832500457763672, 'learning_rate': 1.9989976531650042e-05, 'epoch': 0.07}
{'loss': 1.4938, 'grad_norm': 7.642202854156494, 'learning_rate': 1.9959926220583713e-05, 'epoch': 0.14}
{'loss': 1.3579, 'grad_norm': 3.605273962020874, 'learning_rate': 1.9909909308469398e-05, 'epoch': 0.21}
{'loss': 1.3526, 'grad_norm': 18.823993682861328, 'learning_rate': 1.9840026063894193e-05, 'epoch': 0.29}
{'loss': 1.3796, 'grad_norm': 3.944539785385132, 'learning_rate': 1.9750416581356147e-05, 'epoch': 0.36}
{'loss': 1.346, 'grad_norm': 2.5796878337860107, 'learning_rate': 1.9641260500417672e-05, 'epoch': 0.43}
{'loss': 1.3666, 'grad_norm': 5.86821985244751, 'learning_rate': 1.9512776645583266e-05, 'epoch': 0.5}
{'loss': 1.2946, 'grad_norm': 29.448530197143555, 'learning_rate': 1.9365222587623407e-05, 'epoch': 0.57}
{'loss': 1.3371, 'grad_norm': 4.3936614990234375, 'learning_rate': 1.9198894127224075e-05, 'epoch': 0.64}
{'loss': 1.3334, 'grad_norm': 5.497613906860352, 'lea

...

{'eval_loss': 0.9089449520619204, 'eval_accuracy_emotions': 0.6791, 'eval_accuracy_triggers': 0.8144, 'eval_u_avg_f1': 0.6641, 'eval_u_f1scores_emotions': 0.6276, 'eval_u_f1scores_triggers': 0.7006, 'eval_d_f1scores_emotions': 0.6729, 'eval_d_f1scores_triggers': 0.7778, 'eval_runtime': 688.1643, 'eval_samples_per_second': 4.994, 'eval_steps_per_second': 1.25, 'epoch': 3.0}
{'loss': 0.7764, 'grad_norm': 13.635966300964355, 'learning_rate': 6.527489286891458e-06, 'epoch': 3.06}
{'loss': 0.8378, 'grad_norm': 9.083273887634277, 'learning_rate': 6.111198874944846e-06, 'epoch': 3.14}
{'loss': 0.7524, 'grad_norm': 31.2106876373291, 'learning_rate': 5.702704317997491e-06, 'epoch': 3.21}
{'loss': 0.7624, 'grad_norm': 14.432787895202637, 'learning_rate': 5.302824522501931e-06, 'epoch': 3.28}
{'loss': 0.7926, 'grad_norm': 97.30465698242188, 'learning_rate': 4.912361124952949e-06, 'epoch': 3.35}
{'loss': 0.7403, 'grad_norm': 43.10676574707031, 'learning_rate': 4.532096884851978e-06, 'epoch': 3.42}

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.8097074884282444, 'eval_accuracy_emotions': 0.7425, 'eval_accuracy_triggers': 0.8164, 'eval_u_avg_f1': 0.7044, 'eval_u_f1scores_emotions': 0.7024, 'eval_u_f1scores_triggers': 0.7064, 'eval_d_f1scores_emotions': 0.7451, 'eval_d_f1scores_triggers': 0.7749, 'eval_runtime': 688.7865, 'eval_samples_per_second': 4.99, 'eval_steps_per_second': 1.249, 'epoch': 4.0}
{'loss': 0.6395, 'grad_norm': 9.754467010498047, 'learning_rate': 1.6865479389478545e-06, 'epoch': 4.06}
{'loss': 0.6727, 'grad_norm': 106.34945678710938, 'learning_rate': 1.4461099843816684e-06, 'epoch': 4.13}
{'loss': 0.6378, 'grad_norm': 26.245515823364258, 'learning_rate': 1.2228199589835998e-06, 'epoch': 4.2}
{'loss': 0.6795, 'grad_norm': 19.640962600708008, 'learning_rate': 1.0171254908541373e-06, 'epoch': 4.28}
{'loss': 0.6551, 'grad_norm': 8.009134292602539, 'learning_rate': 8.294389343914899e-07, 'epoch': 4.35}
{'loss': 0.6645, 'grad_norm': 11.364273071289062, 'learning_rate': 6.601365436473439e-07, 'epoch':

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.8072840056462811, 'eval_accuracy_emotions': 0.7664, 'eval_accuracy_triggers': 0.8184, 'eval_u_avg_f1': 0.715, 'eval_u_f1scores_emotions': 0.7386, 'eval_u_f1scores_triggers': 0.6915, 'eval_d_f1scores_emotions': 0.7632, 'eval_d_f1scores_triggers': 0.7775, 'eval_runtime': 688.0235, 'eval_samples_per_second': 4.995, 'eval_steps_per_second': 1.25, 'epoch': 5.0}
{'train_runtime': 11150.5212, 'train_samples_per_second': 12.583, 'train_steps_per_second': 3.146, 'train_loss': 0.9727949126144635, 'epoch': 5.0}


In [14]:
# Training model with seed 77
train_extr_model(77)

c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Training EXTRACTION MODEL with seed 77:


  0%|          | 0/35080 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.5665, 'grad_norm': 18.377416610717773, 'learning_rate': 1.9989976531650042e-05, 'epoch': 0.07}
{'loss': 1.5145, 'grad_norm': 7.188172817230225, 'learning_rate': 1.9959926220583713e-05, 'epoch': 0.14}
{'loss': 1.3948, 'grad_norm': 3.081557273864746, 'learning_rate': 1.9909909308469398e-05, 'epoch': 0.21}
{'loss': 1.4106, 'grad_norm': 20.663869857788086, 'learning_rate': 1.9840026063894193e-05, 'epoch': 0.29}
{'loss': 1.4141, 'grad_norm': 4.097904682159424, 'learning_rate': 1.9750416581356147e-05, 'epoch': 0.36}
{'loss': 1.3724, 'grad_norm': 3.805527925491333, 'learning_rate': 1.9641260500417672e-05, 'epoch': 0.43}
{'loss': 1.3709, 'grad_norm': 6.701464653015137, 'learning_rate': 1.9512776645583266e-05, 'epoch': 0.5}
{'loss': 1.3068, 'grad_norm': 24.170408248901367, 'learning_rate': 1.9365222587623407e-05, 'epoch': 0.57}
{'loss': 1.3509, 'grad_norm': 3.4699249267578125, 'learning_rate': 1.9198894127224075e-05, 'epoch': 0.64}
{'loss': 1.3392, 'grad_norm': 6.089230060577393, 'le

...

{'eval_loss': 0.9640400430791959, 'eval_accuracy_emotions': 0.6223, 'eval_accuracy_triggers': 0.7958, 'eval_u_avg_f1': 0.6176, 'eval_u_f1scores_emotions': 0.554, 'eval_u_f1scores_triggers': 0.6811, 'eval_d_f1scores_emotions': 0.6091, 'eval_d_f1scores_triggers': 0.759, 'eval_runtime': 688.1537, 'eval_samples_per_second': 4.995, 'eval_steps_per_second': 1.25, 'epoch': 3.0}
{'loss': 0.8424, 'grad_norm': 14.6768798828125, 'learning_rate': 6.527489286891458e-06, 'epoch': 3.06}
{'loss': 0.8731, 'grad_norm': 10.729345321655273, 'learning_rate': 6.111198874944846e-06, 'epoch': 3.14}
{'loss': 0.8183, 'grad_norm': 27.23920440673828, 'learning_rate': 5.702704317997491e-06, 'epoch': 3.21}
{'loss': 0.8236, 'grad_norm': 20.687496185302734, 'learning_rate': 5.302824522501931e-06, 'epoch': 3.28}
{'loss': 0.8696, 'grad_norm': 74.28255462646484, 'learning_rate': 4.912361124952949e-06, 'epoch': 3.35}
{'loss': 0.8118, 'grad_norm': 27.098796844482422, 'learning_rate': 4.532096884851978e-06, 'epoch': 3.42}


  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.8707716455246146, 'eval_accuracy_emotions': 0.7181, 'eval_accuracy_triggers': 0.8074, 'eval_u_avg_f1': 0.6752, 'eval_u_f1scores_emotions': 0.658, 'eval_u_f1scores_triggers': 0.6925, 'eval_d_f1scores_emotions': 0.7103, 'eval_d_f1scores_triggers': 0.7705, 'eval_runtime': 688.0853, 'eval_samples_per_second': 4.995, 'eval_steps_per_second': 1.25, 'epoch': 4.0}
{'loss': 0.7047, 'grad_norm': 11.683077812194824, 'learning_rate': 1.6865479389478545e-06, 'epoch': 4.06}
{'loss': 0.7281, 'grad_norm': 62.7010383605957, 'learning_rate': 1.4461099843816684e-06, 'epoch': 4.13}
{'loss': 0.6868, 'grad_norm': 23.74823760986328, 'learning_rate': 1.2228199589835998e-06, 'epoch': 4.2}
{'loss': 0.7344, 'grad_norm': 17.786006927490234, 'learning_rate': 1.0171254908541373e-06, 'epoch': 4.28}
{'loss': 0.7351, 'grad_norm': 6.203692436218262, 'learning_rate': 8.294389343914899e-07, 'epoch': 4.35}
{'loss': 0.7063, 'grad_norm': 13.621657371520996, 'learning_rate': 6.601365436473439e-07, 'epoch': 4.

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.855483139147214, 'eval_accuracy_emotions': 0.7303, 'eval_accuracy_triggers': 0.8173, 'eval_u_avg_f1': 0.6849, 'eval_u_f1scores_emotions': 0.6811, 'eval_u_f1scores_triggers': 0.6887, 'eval_d_f1scores_emotions': 0.7196, 'eval_d_f1scores_triggers': 0.7768, 'eval_runtime': 687.981, 'eval_samples_per_second': 4.996, 'eval_steps_per_second': 1.25, 'epoch': 5.0}
{'train_runtime': 11066.6479, 'train_samples_per_second': 12.679, 'train_steps_per_second': 3.17, 'train_loss': 1.020235451270944, 'epoch': 5.0}


In [15]:
# Training model with seed 111
train_extr_model(111)

c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\marco\anaconda3\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Training EXTRACTION MODEL with seed 111:


  0%|          | 0/35080 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.5486, 'grad_norm': 17.80193328857422, 'learning_rate': 1.9989976531650042e-05, 'epoch': 0.07}
{'loss': 1.4818, 'grad_norm': 8.996939659118652, 'learning_rate': 1.9959926220583713e-05, 'epoch': 0.14}
{'loss': 1.3533, 'grad_norm': 4.325868129730225, 'learning_rate': 1.9909909308469398e-05, 'epoch': 0.21}
{'loss': 1.3473, 'grad_norm': 15.984439849853516, 'learning_rate': 1.9840026063894193e-05, 'epoch': 0.29}
{'loss': 1.374, 'grad_norm': 3.700855016708374, 'learning_rate': 1.9750416581356147e-05, 'epoch': 0.36}
{'loss': 1.3339, 'grad_norm': 2.0504705905914307, 'learning_rate': 1.9641260500417672e-05, 'epoch': 0.43}
{'loss': 1.3508, 'grad_norm': 7.986011505126953, 'learning_rate': 1.9512776645583266e-05, 'epoch': 0.5}
{'loss': 1.277, 'grad_norm': 34.078277587890625, 'learning_rate': 1.9365222587623407e-05, 'epoch': 0.57}
{'loss': 1.3119, 'grad_norm': 4.179061412811279, 'learning_rate': 1.9198894127224075e-05, 'epoch': 0.64}
{'loss': 1.2988, 'grad_norm': 5.24502420425415, 'learni

...

{'eval_loss': 0.8320291438148364, 'eval_accuracy_emotions': 0.691, 'eval_accuracy_triggers': 0.8176, 'eval_u_avg_f1': 0.6756, 'eval_u_f1scores_emotions': 0.6467, 'eval_u_f1scores_triggers': 0.7045, 'eval_d_f1scores_emotions': 0.6702, 'eval_d_f1scores_triggers': 0.7785, 'eval_runtime': 688.0174, 'eval_samples_per_second': 4.996, 'eval_steps_per_second': 1.25, 'epoch': 3.0}
{'loss': 0.7316, 'grad_norm': 20.102375030517578, 'learning_rate': 6.527489286891458e-06, 'epoch': 3.06}
{'loss': 0.7868, 'grad_norm': 9.32190227508545, 'learning_rate': 6.111198874944846e-06, 'epoch': 3.14}
{'loss': 0.6971, 'grad_norm': 45.53240966796875, 'learning_rate': 5.702704317997491e-06, 'epoch': 3.21}
{'loss': 0.6974, 'grad_norm': 8.432028770446777, 'learning_rate': 5.302824522501931e-06, 'epoch': 3.28}
{'loss': 0.7387, 'grad_norm': 75.95780181884766, 'learning_rate': 4.912361124952949e-06, 'epoch': 3.35}
{'loss': 0.6739, 'grad_norm': 36.87929916381836, 'learning_rate': 4.532096884851978e-06, 'epoch': 3.42}
{

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.7570977134885727, 'eval_accuracy_emotions': 0.7701, 'eval_accuracy_triggers': 0.819, 'eval_u_avg_f1': 0.7207, 'eval_u_f1scores_emotions': 0.7378, 'eval_u_f1scores_triggers': 0.7036, 'eval_d_f1scores_emotions': 0.7611, 'eval_d_f1scores_triggers': 0.7776, 'eval_runtime': 688.2168, 'eval_samples_per_second': 4.994, 'eval_steps_per_second': 1.25, 'epoch': 4.0}
{'loss': 0.5883, 'grad_norm': 9.0853910446167, 'learning_rate': 1.6865479389478545e-06, 'epoch': 4.06}
{'loss': 0.6092, 'grad_norm': 44.74871063232422, 'learning_rate': 1.4461099843816684e-06, 'epoch': 4.13}
{'loss': 0.5904, 'grad_norm': 16.358474731445312, 'learning_rate': 1.2228199589835998e-06, 'epoch': 4.2}
{'loss': 0.6186, 'grad_norm': 14.376166343688965, 'learning_rate': 1.0171254908541373e-06, 'epoch': 4.28}
{'loss': 0.605, 'grad_norm': 7.927257061004639, 'learning_rate': 8.294389343914899e-07, 'epoch': 4.35}
{'loss': 0.5805, 'grad_norm': 17.53046417236328, 'learning_rate': 6.601365436473439e-07, 'epoch': 4.42}

  0%|          | 0/860 [00:00<?, ?it/s]

{'eval_loss': 0.7610925864617913, 'eval_accuracy_emotions': 0.7847, 'eval_accuracy_triggers': 0.8228, 'eval_u_avg_f1': 0.7281, 'eval_u_f1scores_emotions': 0.7618, 'eval_u_f1scores_triggers': 0.6944, 'eval_d_f1scores_emotions': 0.7713, 'eval_d_f1scores_triggers': 0.7819, 'eval_runtime': 688.0019, 'eval_samples_per_second': 4.996, 'eval_steps_per_second': 1.25, 'epoch': 5.0}
{'train_runtime': 11063.3541, 'train_samples_per_second': 12.682, 'train_steps_per_second': 3.171, 'train_loss': 0.9220450930883566, 'epoch': 5.0}


Train freezed model

In [ ]:
# Training model with seed 49
train_extr_model(49, True)

In [ ]:
# Training model with seed 666
train_extr_model(666, True)

In [ ]:
# Training model with seed 51
train_extr_model(51, True)

In [ ]:
# Training model with seed 77
train_extr_model(77, True)

In [ ]:
# Training model with seed 111
train_extr_model(111, True)